一、任务说明

本文件代码展示ResNet101局部表示+LSTM+注意力机制模型训练过程。

二、实验数据（参见getdata.py说明）

三、实验环境


硬件要求： CPU或支持GPU加速的硬件设备，如NVIDIA GPU，用于加速深度学习模型的训练和推理。

操作系统：Windows10

编写环境：jupyter notebook

深度学习框架：依赖于深度学习框架PyTorch


四、所用的方法

1、使用ResNet101局部表示+LSTM+注意力机制模型训练实现。（模型详细说明参见models.py）


相比vgg19整体表示+GRU模型改进：

（1）图像编码器选择预训练的ResNet101，解码器选择LSTM。

①ResNet101相较vgg19的优势在于：增加了输入到输出的直连，在输入和输出时都做了BN，能够缓解vgg19网络深带来的梯度消失难优化问题，提高训练速度的同时能够支持更高的学习率。

②LSTM在数据相对充足的情况下表征能力较GRU更好。

（2）引入微调，减小计算量，加快模型训练。

①默认微调ResNet101的2-4个卷积块，相比以小学习率调整整个vgg19模型，计算量小，编码可靠性、准确性更高，能更快适应本例给出的新的编码场景。

②默认微调词嵌入层。

（3）使用参照dropout值的随机监督模式选择。

每次训练产生随机值，当该随机值小于给定的dropout值，则使用有监督学习。

（4）引入学习率衰减以更好地适应训练，防止模型过拟合“死掉”。


In [1]:
import torch
import nltk
nltk.download('punkt')

import torchvision.transforms as transforms
import torch.nn as nn
from models import Encoder, DecoderWithAttention
from datasets import *
from solver import *
import torch.backends.cudnn as cudnn
from getdata import words,get_sequence,mktrainval
cudnn.benchmark = True  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chubby\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
D:\ProgramData\Anaconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'D:\ProgramData\Anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chubby\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chubby\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2、超参数说明：
词嵌入的维度    'embed_dim' : 256
注意力机制线性层的维度  'attention_dim' :256
解码器 RNN 的维度   'decoder_dim' : 256
丢弃率  'dropout' : 0.5
批量大小    'batch_size' : 8
编码器的学习率  'encoder_lr' : 1e-4
解码器的学习率  'decoder_lr' : 4e-4
梯度剪裁的绝对值阈值    'grad_clip' : 5.
'doubly stochastic attention' 的正则化参数  'alpha_c' : 1.

In [2]:
#这是一个配置字典，包含了训练过程中所需的各种参数。

cfg = {
    # Data parameters
    'train_json_path': r'C:\Users\chubby\Desktop\训练测试划分\train_captions.json',
    'test_json_path': r'C:\Users\chubby\Desktop\训练测试划分\test_captions.json',
    'image_folder' : r'C:\Users\chubby\Desktop\code\images', 
    'data_name' : 'DeepFashion_MultiModal',
    # Model parameters
    'embed_dim' : 256,  # dimension of word embeddings     # 词嵌入的维度
    'attention_dim' :256,  # dimension of attention linear layers   # 注意力机制线性层的维度
    'decoder_dim' : 256,  # dimension of decoder RNN  # 解码器 RNN 的维度
    'dropout' : 0.5,  # 丢弃率
    'device' : torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),  # sets device for model and PyTorch tensors 
    # 设置模型和 PyTorch 张量的设备
    
    # Training parameters
    'start_epoch' : 0,# 开始的轮次
    'epochs' : 10,  # number of epochs to train for (if early stopping is not triggered)# 训练轮次
    'epochs_since_improvement' : 0,  # keeps track of number of epochs since there's been an improvement in validation BLEU
    # 记录自上次验证 BLEU 有所改善以来的轮次数
    'batch_size' : 8,# 批量大小
    'workers' : 1,  # for data-loading; right now, only 1 works  # 用于数据加载的工作线程数（当前只有 1 有效）
    'encoder_lr' : 1e-4,  # learning rate for encoder if fine-tuning  # 如果进行微调，编码器的学习率
    'decoder_lr' : 4e-4,  # learning rate for decoder  # 解码器的学习率
    'grad_clip' : 5.,  # clip gradients at an absolute value of # 梯度剪裁的绝对值阈值
    'alpha_c' : 1.,  # regularization parameter for 'doubly stochastic attention', as in the paper
    # 'doubly stochastic attention' 的正则化参数
    'best_bleu4' : 0.,  # BLEU-4 score right now # 当前的 BLEU-4 分数
    'print_freq' : 50,  # print training/validation stats every __ batches  # 每 __ 个批次打印训练/验证统计信息
    'fine_tune_encoder' : False,  # fine-tune encoder or not  # 是否微调编码器
    'checkpoint' : None,  # path to checkpoint, None if none # 检查点路径，如果没有则为 None
}

In [3]:
word_map = words(cfg['train_json_path'])# 从训练集的 JSON 文件中获取单词映射
cfg['vocab_size'] = len(word_map)# 将词汇表大小添加到配置中
batch_size = cfg['batch_size']
# 获取批量大小和数据加载器
train_loader, val_loader = mktrainval(cfg['train_json_path'], 
                                                 cfg['test_json_path'], 
                                                 word_map, batch_size, 
                                                 workers=cfg['workers'],
                                                 file_path=cfg['image_folder'])
# 以下是对函数 `mktrainval` 的一些假设的注释
# 该函数的目的是创建训练和验证数据加载器，返回两个 DataLoader 对象

# for image, caption, caplen in train_loader:
#     print(image, caption, caplen)
#     break
# 上述代码是为了展示一批次数据的示例，可以用于检查数据加载器是否正常工作
# 实际训练时，会在训练循环中使用这些数据加载器

In [4]:
if cfg['checkpoint'] is None:
     # 如果没有提供检查点文件，创建新的编码器和解码器
    encoder = Encoder()
    encoder.fine_tune(cfg['fine_tune_encoder'])
    encoder_optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, encoder.parameters()), # 是否对编码器进行微调
    encoder_optimizer = (
                                         lr=cfg['encoder_lr']) if cfg['fine_tune_encoder'] else None
    decoder = DecoderWithAttention(cfg)
    decoder_optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, decoder.parameters()),
                                         lr=cfg['decoder_lr'])
else:
                                         
     # 如果提供了检查点文件，加载之前训练的模型和优化器状态
    checkpoint = torch.load(cfg['checkpoint'])
    cfg['start_epoch'] = checkpoint['epoch'] + 1
    cfg['epochs_since_improvement'] = checkpoint['epochs_since_improvement']
    cfg['best_bleu4'] = checkpoint['bleu-4']
    encoder = checkpoint['encoder']
    encoder_optimizer = checkpoint['encoder_optimizer']
    decoder = checkpoint['decoder']
    decoder_optimizer = checkpoint['decoder_optimizer']
                                         
    # 如果需要对编码器进行微调而且优化器为空，则创建一个新的编码器优化器
    if cfg['fine_tune_encoder'] is True and encoder_optimizer is None:
        encoder.fine_tune(cfg['fine_tune_encoder'])
        encoder_optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, encoder.parameters()),
                                             lr=cfg['encoder_lr'])

In [5]:
# Move to GPU, if available
decoder = decoder.to(cfg['device'])
encoder = encoder.to(cfg['device'])


3、损失函数说明：

criterion = nn.CrossEntropyLoss().to(cfg['device'])

这行代码创建了一个交叉熵损失函数的实例，并将其移动到配置中指定的设备（GPU 或 CPU）。

然后使用交叉熵损失函数计算模型生成的概率分布与实际标注的概率分布之间的差异。

nn.CrossEntropyLoss 通常用于多类别分类任务，其输入是模型的输出分数和实际目标类别的索引。在图像标注任务中，这个损失函数常用于计算模型生成的单词序列和实际标注之间的差异。


In [6]:
# Loss function
criterion = nn.CrossEntropyLoss().to(cfg['device'])
#这行代码创建了一个交叉熵损失函数的实例，并将其移动到配置中指定的设备（GPU 或 CPU）。
# 然后使用交叉熵损失函数计算模型生成的概率分布与实际标注的概率分布之间的差异。
# nn.CrossEntropyLoss 通常用于多类别分类任务，其输入是模型的输出分数和实际目标类别的索引。在图像标注任务中，这个损失函数常用于计算模型生成的单词序列和实际标注之间的差异。

五、实验结果

In [ ]:
# Epochs
for epoch in range(cfg['start_epoch'], cfg['epochs']):
    
     # 如果连续8个epoch没有改进，学习率衰减，并在20个epoch没有改进时终止训练
    # Decay learning rate if there is no improvement for 8 consecutive epochs, and terminate training after 20
    if cfg['epochs_since_improvement'] == 20:
        break
    if cfg['epochs_since_improvement'] > 0 and cfg['epochs_since_improvement'] % 8 == 0:
        adjust_learning_rate(decoder_optimizer, 0.8)
        if cfg['fine_tune_encoder']:
            adjust_learning_rate(encoder_optimizer, 0.8)

      # 训练一个epoch
    # One epoch's training
    train(train_loader=train_loader,
          encoder=encoder,
          decoder=decoder,
          criterion=criterion,
          encoder_optimizer=encoder_optimizer,
          decoder_optimizer=decoder_optimizer,
          epoch=epoch,
          cfg=cfg)
    
    # 验证一个epoch
    # One epoch's validation
    validate(val_loader=val_loader,
                            encoder=encoder,
                            decoder=decoder,
                            criterion=criterion,
                            word_map=word_map,
                            cfg=cfg)

      # 保存检查点
    # Save checkpoint
    save_checkpoint(cfg['data_name'], epoch, cfg['epochs_since_improvement'], encoder, decoder, encoder_optimizer,
                    decoder_optimizer, 0, True)

Epoch: [0][0/1270]	Batch Time 20.083 (20.083)	Data Load Time 17.366 (17.366)	Loss 5.7583 (5.7583)	Top-5 Accuracy 2.114 (2.114)
Epoch: [0][50/1270]	Batch Time 0.387 (0.756)	Data Load Time 0.000 (0.341)	Loss 4.7605 (5.0050)	Top-5 Accuracy 35.729 (29.523)
Epoch: [0][100/1270]	Batch Time 0.373 (0.562)	Data Load Time 0.000 (0.172)	Loss 4.5885 (4.8537)	Top-5 Accuracy 39.565 (33.278)
Epoch: [0][150/1270]	Batch Time 0.369 (0.499)	Data Load Time 0.000 (0.115)	Loss 4.0547 (4.6818)	Top-5 Accuracy 55.381 (37.741)
Epoch: [0][200/1270]	Batch Time 0.388 (0.469)	Data Load Time 0.001 (0.087)	Loss 3.7992 (4.4576)	Top-5 Accuracy 63.441 (43.863)
Epoch: [0][250/1270]	Batch Time 0.373 (0.451)	Data Load Time 0.000 (0.069)	Loss 3.1870 (4.2209)	Top-5 Accuracy 76.549 (49.845)
Epoch: [0][300/1270]	Batch Time 0.393 (0.439)	Data Load Time 0.000 (0.058)	Loss 3.4604 (4.0202)	Top-5 Accuracy 69.679 (54.716)
Epoch: [0][350/1270]	Batch Time 0.374 (0.430)	Data Load Time 0.000 (0.050)	Loss 2.6619 (3.8507)	Top-5 Accuracy 8

六、实验结果分析

Epoch 0:
Loss: 初始损失为5.7583，并随着训练的进行逐渐减小，最终下降到2.7512。这表明模型在训练数据上逐渐学到了任务的模式。
Top-5 Accuracy: 初始 Top-5 Accuracy 为2.114%，随着训练逐渐提高，最终达到了81.881%。这说明模型在前五个预测中包含正确标签的概率在训练过程中不断提高。
Epoch 1:
Loss: 新的 epoch 开始，损失重新升高（2.7512），然后逐渐减小到2.0404。这是因为模型继续学习并逐渐逼近更优解。
Top-5 Accuracy: 随着训练的进行，Top-5 Accuracy 从81.881% 提高到94.728%。这表示模型在训练数据上的性能不断提升。

Loss 变化： 损失逐渐减小表明模型在学习任务的过程中在一定程度上取得了成功。然而，如果在后续 epoch 中损失再次升高，可能需要考虑调整学习率或其他训练参数。

Top-5 Accuracy 变化： Top-5 Accuracy 在训练过程中不断提高，这是一个良好的迹象。这意味着模型能够在前五个预测中更准确地包含正确标签。如果这一趋势持续，可以考虑增加训练周期以进一步提高性能。

七、总结


1、实现功能：

（1）encoder： 通过CNN局部表示resnet101倒数特征层编码图像

（2）decoder：通过RNN自注意力机制模型解码生成图像的描述性文本

（3）Beam Search ： 采用了 Beam Search 解码方法来生成图像描述，这有助于获得更准确和多样的描述

2、创新点：

（1）Attention 机制： 代码中使用了 Attention 机制，能够使模型在生成描述时更加关注图像的不同部分。

（2）相较vgg19整体表示+GRU模型的改善：在反思上一个模型的基础上，尝试新的编码器、解码器选择，针对编码器的预训练模型和词嵌入层，引入微调的策略，同时在训练过程中引入学习率衰减和基于dropout的随机监督模式选择，提升了模型的表征能力。

3、不足与反思：

（1）Overfitting 风险： 需要关注模型是否存在过拟合的风险，特别是在训练集上的表现很好，但在验证集上性能较差时。

（2）训练参数调优： 日志中没有提到是否进行了超参数的调优。调整学习率、批量大小等参数可能会对模型性能产生重要影响。

（3）模型解释性： 如果模型是黑盒的，难以解释模型如何生成描述。有时候，理解模型的决策是非常重要的。

4、反思与改进方向：

（1）监控和可视化： 在训练过程中添加更多的监控和可视化，例如绘制损失曲线、生成的描述示例等，有助于更好地了解模型的训练过程。

（2）超参数搜索： 尝试不同的超参数组合，使用交叉验证等方法来找到最佳的模型配置。

（3）模型评估： 使用更多的评价指标来全面评估模型性能，如BLEU、METEOR等，以更全面地了解模型生成描述的质量。
